# Machine Learning Model (EMMA)

### This web application aims to tell you where you should roll your business out in Mexico, based on demographic information (income, location, education, and more)


# ETL process

In [2]:
pip install geopandas

  Running setup.py clean for fiona
Failed to build fiona
    Running setup.py install for fiona: started
    Running setup.py install for fiona: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


    ERROR: Command errored out with exit status 1:
     command: 'D:\Users\rodrigo.guarneros\Anaconda4\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'D:\\Users\\rodrigo.guarneros\\AppData\\Local\\Temp\\pip-install-mha8smgt\\fiona_ab26aab3639e418c96b9756ea5b10a81\\setup.py'"'"'; __file__='"'"'D:\\Users\\rodrigo.guarneros\\AppData\\Local\\Temp\\pip-install-mha8smgt\\fiona_ab26aab3639e418c96b9756ea5b10a81\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'D:\Users\rodrigo.guarneros\AppData\Local\Temp\pip-pip-egg-info-xd1zls_6'
         cwd: D:\Users\rodrigo.guarneros\AppData\Local\Temp\pip-install-mha8smgt\fiona_ab26aab3639e418c96b9756ea5b10a81\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment var

In [2]:
!pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install descartes

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import dependencies
import pandas as pd
import re
import json
import pymongo
#import geopandas as gpd
import matplotlib.pyplot as plt
import descartes
import os

In [36]:
# Read data
censo = pd.read_csv("Censo_2020_final.csv", encoding='Latin1', low_memory=False)
deposits = pd.read_csv("deposits.csv", encoding='Latin1', low_memory=False)
infrastructure = pd.read_csv("financial_infrastructure.csv", encoding='Latin1', low_memory=False)
use = pd.read_csv("financial_use.csv", encoding='Latin1', low_memory=False)


In [37]:
censo.columns

Index(['ï»¿ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'LONGITUD',
       'LATITUD', 'ALTITUD', 'POBTOT', 'POBFEM', 'POBMAS', 'POB_TOT_6more',
       'POB_TOT_6more_F', 'POB_TOT_6more_M', 'GRAPROES', 'GRAPROES_F',
       'GRAPROES_M', 'POCUPADA', 'POCUPADA_F', 'POCUPADA_M', 'TVIVHAB',
       'VPH_C_ELEC', 'VPH_AGUADV', 'VPH_DRENAJ', 'VPH_REFRI', 'VPH_AUTOM',
       'VPH_MOTO', 'VPH_RADIO', 'VPH_TV', 'VPH_PC', 'VPH_TELEF', 'VPH_CEL',
       'VPH_INTER', 'VPH_STVP', 'VPH_SPMVPI', 'VPH_CVJ', 'TAMLOC',
       'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41',
       'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45',
       'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49',
       'Unnamed: 50'],
      dtype='object')

In [38]:
censo = censo.loc[:, ~censo.columns.str.contains('^Unnamed')]
censo.columns

Index(['ï»¿ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'LONGITUD',
       'LATITUD', 'ALTITUD', 'POBTOT', 'POBFEM', 'POBMAS', 'POB_TOT_6more',
       'POB_TOT_6more_F', 'POB_TOT_6more_M', 'GRAPROES', 'GRAPROES_F',
       'GRAPROES_M', 'POCUPADA', 'POCUPADA_F', 'POCUPADA_M', 'TVIVHAB',
       'VPH_C_ELEC', 'VPH_AGUADV', 'VPH_DRENAJ', 'VPH_REFRI', 'VPH_AUTOM',
       'VPH_MOTO', 'VPH_RADIO', 'VPH_TV', 'VPH_PC', 'VPH_TELEF', 'VPH_CEL',
       'VPH_INTER', 'VPH_STVP', 'VPH_SPMVPI', 'VPH_CVJ', 'TAMLOC'],
      dtype='object')

In [39]:
censo = censo.rename(columns={'ï»¿ENTIDAD':'Entidad'}, inplace=False)
censo.columns

Index(['Entidad', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'LONGITUD',
       'LATITUD', 'ALTITUD', 'POBTOT', 'POBFEM', 'POBMAS', 'POB_TOT_6more',
       'POB_TOT_6more_F', 'POB_TOT_6more_M', 'GRAPROES', 'GRAPROES_F',
       'GRAPROES_M', 'POCUPADA', 'POCUPADA_F', 'POCUPADA_M', 'TVIVHAB',
       'VPH_C_ELEC', 'VPH_AGUADV', 'VPH_DRENAJ', 'VPH_REFRI', 'VPH_AUTOM',
       'VPH_MOTO', 'VPH_RADIO', 'VPH_TV', 'VPH_PC', 'VPH_TELEF', 'VPH_CEL',
       'VPH_INTER', 'VPH_STVP', 'VPH_SPMVPI', 'VPH_CVJ', 'TAMLOC'],
      dtype='object')

In [40]:
censo

,Entidad,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_RADIO,VPH_TV,VPH_PC,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,TAMLOC
0,1,Aguascalientes,1,Aguascalientes,0,Total del Municipio,NaN,NaN,NaN,948990,...,221315,257061,136923,116647,251719,178619,130290,80951,56131,*
1,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,"102Â°17'45.768"" W","21Â°52'47.362"" N",1878,863893,...,204535,237274,130384,112002,232793,169675,123670,77719,53589,13
2,1,Aguascalientes,1,Aguascalientes,94,Granja Adelita,"102Â°22'24.710"" W","21Â°52'18.749"" N",1902,5,...,*,*,*,*,*,*,*,*,*,1
3,1,Aguascalientes,1,Aguascalientes,96,Agua Azul,"102Â°21'25.639"" W","21Â°53'01.522"" N",1861,41,...,11,11,5,2,11,4,5,2,1,1
4,1,Aguascalientes,1,Aguascalientes,102,Los Arbolitos [Rancho],"102Â°21'26.261"" W","21Â°46'48.650"" N",1861,8,...,*,*,*,*,*,*,*,*,*,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98402,32,Zacatecas,58,Santa MarÃ­a de la Paz,37,Mesa Grande,"103Â°24'52.925"" W","21Â°33'12.375"" N",1944,165,...,29,44,6,29,24,9,13,0,0,1
98403,32,Zacatecas,58,Santa MarÃ­a de la Paz,39,San Isidro,"103Â°20'14.164"" W","21Â°29'11.130"" N",2031,1,...,*,*,*,*,*,*,*,*,*,1
98404,32,Zacatecas,58,Santa MarÃ­a de la Paz,41,San Miguel TepetitlÃ¡n,"103Â°20'09.356"" W","21Â°30'15.168"" N",1977,97,...,20,26,4,1,21,12,21,1,1,1
98405,32,Zacatecas,58,Santa MarÃ­a de la Paz,42,San Rafael,"103Â°22'20.134"" W","21Â°31'39.341"" N",2042,3,...,*,*,*,*,*,*,*,*,*,1


In [32]:
infrastructure

,Clave Municipio,Clave \nEstado,Región,Estado,Municipio,Tipo de población,Branch_BC,Branch_DB,Sucursales Socap,Sucursales Sofipo,Branch_Total,Corresponsales,ATM,PST,Building_PST,MB_contracts
0,1001,1,Occidente y Bajío,Aguascalientes,Aguascalientes,Semi-metrópoli,117,2,16,8,143,458,611,17570,11424,393411
1,1002,1,Occidente y Bajío,Aguascalientes,Asientos,Urbano,0,0,0,0,0,8,4,68,54,8717
2,1003,1,Occidente y Bajío,Aguascalientes,Calvillo,Urbano,5,0,2,0,7,19,17,152,136,11549
3,1004,1,Occidente y Bajío,Aguascalientes,Cosío,Semi-urbano,0,0,0,0,0,5,4,27,21,2044
4,1005,1,Occidente y Bajío,Aguascalientes,Jesús María,Urbano,3,0,1,0,4,48,23,828,654,23794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2461,32055,32,Occidente y Bajío,Zacatecas,Villanueva,Semi-urbano,3,0,0,0,3,8,3,39,32,4792
2462,32056,32,Occidente y Bajío,Zacatecas,Zacatecas,Urbano,27,1,6,3,37,112,163,3446,2337,93016
2463,32057,32,Occidente y Bajío,Zacatecas,Trancoso,Semi-urbano,0,0,0,0,0,2,2,16,8,2635
2464,32058,32,Occidente y Bajío,Zacatecas,Santa María de la Paz,Rural,0,0,0,0,0,0,0,2,2,77


# Transform
- Census

In [33]:
# First, we remove all the blank values from rows that sum up total, since they don't have any latitude or longitude we can use
censo = censo.dropna()
censo

,Entidad,NOM_ENT,MUN,NOM_MUN,LOC,NOM_LOC,LONGITUD,LATITUD,ALTITUD,POBTOT,...,VPH_RADIO,VPH_TV,VPH_PC,VPH_TELEF,VPH_CEL,VPH_INTER,VPH_STVP,VPH_SPMVPI,VPH_CVJ,TAMLOC
1,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,"102Â°17'45.768"" W","21Â°52'47.362"" N",1878,863893,...,204535,237274,130384,112002,232793,169675,123670,77719,53589,13
2,1,Aguascalientes,1,Aguascalientes,94,Granja Adelita,"102Â°22'24.710"" W","21Â°52'18.749"" N",1902,5,...,*,*,*,*,*,*,*,*,*,1
3,1,Aguascalientes,1,Aguascalientes,96,Agua Azul,"102Â°21'25.639"" W","21Â°53'01.522"" N",1861,41,...,11,11,5,2,11,4,5,2,1,1
4,1,Aguascalientes,1,Aguascalientes,102,Los Arbolitos [Rancho],"102Â°21'26.261"" W","21Â°46'48.650"" N",1861,8,...,*,*,*,*,*,*,*,*,*,1
5,1,Aguascalientes,1,Aguascalientes,104,Ardillas de Abajo (Las Ardillas),"102Â°11'30.914"" W","21Â°56'42.243"" N",1989,1,...,*,*,*,*,*,*,*,*,*,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98401,32,Zacatecas,58,Santa MarÃ­a de la Paz,35,Los Trigos (Mesa de los Trigos),"103Â°14'27.792"" W","21Â°29'51.355"" N",2499,89,...,13,18,2,0,19,1,5,0,0,1
98402,32,Zacatecas,58,Santa MarÃ­a de la Paz,37,Mesa Grande,"103Â°24'52.925"" W","21Â°33'12.375"" N",1944,165,...,29,44,6,29,24,9,13,0,0,1
98403,32,Zacatecas,58,Santa MarÃ­a de la Paz,39,San Isidro,"103Â°20'14.164"" W","21Â°29'11.130"" N",2031,1,...,*,*,*,*,*,*,*,*,*,1
98404,32,Zacatecas,58,Santa MarÃ­a de la Paz,41,San Miguel TepetitlÃ¡n,"103Â°20'09.356"" W","21Â°30'15.168"" N",1977,97,...,20,26,4,1,21,12,21,1,1,1


In [34]:
# Then, create a function to change the coordinates to the correct type
def dms2dd(s):
    # example: s = """0°51'56.29"S"""
    degrees, minutes, seconds, direction = re.split('[°\'"]+', s)
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction in ('S','W'):
        dd*= -1
    return dd


In [35]:
# We use the function to change both latitude and longitude columns
censo['LATITUD'] = censo['LATITUD'].apply(dms2dd)
censo['LONGITUD'] = censo['LONGITUD'].apply(dms2dd)*-1

ValueError: could not convert string to float: '21Â'